<a href="https://colab.research.google.com/github/ProtossDragoon/paper_implementation_and_testing_tf2/blob/main/utils/rosutils/Convert_Bagfile_to_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert ROS Bagfile to Video

## Author

name : Janghoo Lee <br>
github : https://github.com/ProtossDragoon <br>
contact : dlwkdgn1@naver.com <br>
circle : https://github.com/sju-coml <br>
organization : https://web.deering.co/ <br>
published date : June, 2021


## Related Notebook

[Notebooks](https://github.com/ProtossDragoon/paper_implementation_and_testing_tf2/tree/main/notebooks)


# Environment

In [35]:
import os, sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm as tqdm

## Install ROS (Melodic)

In [36]:
!sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'

!sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654

!sudo apt update

!sudo apt install ros-melodic-desktop-full

!pip install --extra-index-url https://rospypi.github.io/simple/ rospy

import rospy
### wait, we need a master:

!printf -- '#!/bin/bash\n' > ros_start.sh
!printf -- 'source /opt/ros/melodic/setup.bash\n' >> ros_start.sh
!printf -- '/opt/ros/melodic/bin/roscore' >> ros_start.sh
!chmod +x ros_start.sh
!cat ros_start.sh

import subprocess
import os
prc = subprocess.Popen([os.getcwd()+"/ros_start.sh"])

Executing: /tmp/apt-key-gpghome.T7kTrk0jyG/gpg.1.sh --keyserver hkp://keyserver.ubuntu.com:80 --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
gpg: key F42ED6FBAB17C654: "Open Robotics <info@osrfoundation.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://packages.ros.org/ros/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.

## Google Drive / Git

In [37]:
#!rm -r /content/gdrive/
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
HOME_DIR = "/content/gdrive/MyDrive"
WS = os.path.join(HOME_DIR, "ColabWorkspace")
DATA_DIR = os.path.join(HOME_DIR, 'data')
DATA_ROSBAG_DIR = os.path.join(DATA_DIR, 'rosbag')
DATA_VIDEO_DIR = os.path.join(DATA_DIR, 'video')

GIT_REPO_NAME = 'paper_implementation_and_testing_tf2'
GIT_WS = os.path.join(WS, GIT_REPO_NAME)

GIT_BRANCH = 'master'
GIT_USERNAME = None # Add herea
GIT_EMAIL = None
GIT_PASSWORD = None # Add here

In [39]:
!mkdir -p {WS}
%cd {WS}
!git clone https://github.com/ProtossDragoon/{GIT_REPO_NAME}.git
%cd {GIT_WS}
!git config --global user.name {GIT_USERNAME}
!git config --global user.email {GIT_EMAIL}
!git checkout -b {GIT_BRANCH}
!git checkout {GIT_BRANCH}
!git pull origin {GIT_BRANCH}
!git push origin {GIT_BRANCH}
!git branch
!git branch --set-upstream-to origin/{GIT_BRANCH}

/content/gdrive/MyDrive/ColabWorkspace
fatal: destination path 'paper_implementation_and_testing_tf2' already exists and is not an empty directory.
/content/gdrive/MyDrive/ColabWorkspace/paper_implementation_and_testing_tf2
fatal: A branch named 'master' already exists.
Already on 'master'
fatal: Couldn't find remote ref master
fatal: could not read Username for 'https://github.com': No such device or address
  list
  main
* master
  resnet
  tftrt
  unet
error: the requested upstream branch 'origin/master' does not exist
hint: 
hint: If you are planning on basing your work on an upstream
hint: branch that already exists at the remote, you may need to
hint: run "git fetch" to retrieve it.
hint: 
hint: If you are planning to push out a new local branch that
hint: will track its remote counterpart, you may want to use
hint: "git push -u" to set the upstream config as you push.


# Convert

## Hyper Parameters

In [42]:
DATA_BAGFILE_NAME = "ku1.bag" #@param {type:"string"}
DATA_BAGFILE_PATH  = os.path.join(DATA_ROSBAG_DIR, DATA_BAGFILE_NAME)#@param
OUTPUT_PATH = os.path.join(DATA_VIDEO_DIR, DATA_BAGFILE_NAME) #@param

print('{} -> {}'.format(DATA_BAGFILE_PATH, OUTPUT_PATH))

/content/gdrive/MyDrive/data/rosbag/ku1.bag -> /content/gdrive/MyDrive/data/video/ku1.bag


## Run

In [ ]:
print(bagfile)
outfile = args.outfile

if not outfile:
    outfile = os.path.join(*os.path.split(bagfile)[-1].split('.')[:-1]) + '.avi'

bag = rosbag.Bag(bagfile, 'r')
print('Calculating video properties')
rate, minrate, maxrate, size, times = get_info(bag, args.topic, start_time=args.start, stop_time=args.end)
nframes = calc_n_frames(times, args.precision)
# writer = cv2.VideoWriter(outfile, cv2.cv.CV_FOURCC(*'DIVX'), rate, size)
writer = cv2.VideoWriter(outfile, cv2.VideoWriter_fourcc(*'DIVX'), np.ceil(maxrate*args.precision), size)
print('Writing video')
write_frames(bag, writer, len(times), topic=args.topic, nframes=nframes, start_time=args.start, stop_time=args.end, encoding=args.encoding)
writer.release()
print('\n')